In [ ]:
##### This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
count = 0
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames: 
        count += 1
        #print(filename)
print("There are " + count + " files in the input directory")
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

: 

In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt

plt.figure(figsize = (5,5))

img = cv.imread('/kaggle/input/ships-in-satellite-imagery/shipsnet/shipsnet/1__20170618_180801_0f34__-122.34834288305811_37.74061313831539.png')
# img = cv.imread('/kaggle/input/ships-in-satellite-imagery/shipsnet/shipsnet/0__20150718_184300_090b__-122.35324421973536_37.772113980272394.png')
plt.imshow(img)


In [ ]:
imggray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
plt.imshow(imggray)

In [ ]:
ret, thresh = cv.threshold(imggray,0,255,cv.THRESH_BINARY + cv.THRESH_OTSU)
plt.imshow(thresh)

In [ ]:
kernel = np.ones((3,3),np.uint8)
opening = cv.morphologyEx(thresh,cv.MORPH_OPEN,kernel)
plt.imshow(opening)
# noise removal, this is kinda useless here cos thresh seems to have done the job well enough

In [ ]:
kernel = np.ones((3,3),np.uint8)
opening_grad = cv.morphologyEx(thresh,cv.MORPH_GRADIENT,kernel,iterations= 1)
plt.imshow(opening_grad)
# difference between erosion and dilation -> gives the outline

In [ ]:
sure_bg = cv.dilate(opening, kernel, iterations=3)
plt.imshow(sure_bg)

In [ ]:

dist_transform = cv.distanceTransform(opening,cv.DIST_L2,5)
dist_transform = cv.normalize(dist_transform, None, 0, 1.0, cv.NORM_MINMAX)

ret, sure_fg = cv.threshold(dist_transform, 0.6 * dist_transform.max(), 255, 0)
plt.imshow(sure_fg)


In [ ]:
sure_fg = np.uint8(sure_fg)
ret, markers = cv.connectedComponents(sure_fg)



In [ ]:
plt.imshow(markers)

In [ ]:
markers += 1
plt.imshow(markers)

In [ ]:
unknown = cv.subtract(sure_bg, sure_fg)
markers[unknown==255] = 0
plt.imshow(markers)

In [ ]:
markers = cv.watershed(img,markers)
img[markers == -1] = [255,0,0]

plt.imshow( img)

In [ ]:
def measure_ships(markers, image):
    # Initialize a list to store ship dimensions
    ship_dimensions = []
    for marker in np.unique(markers):
        if marker == -1 or marker == 1:
            continue
        mask = np.zeros_like(image, dtype=np.uint8)
        mask[markers == marker] = 255
        contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        for contour in contours:
            x, y, w, h = cv.boundingRect(contour)
            length = max(w, h)
            width = min(w, h)
            ship_dimensions.append((length, width))
    return ship_dimensions


 

In [ ]:
_, mask = cv.threshold(imggray, 127, 255, cv.THRESH_BINARY)
plt.imshow(mask)

In [ ]:
contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
# print(contours)

In [ ]:
ship_dimensions = []
for c in contours:
        x, y, w, h = cv.boundingRect(c)
        length= max(w,h)
        width = min(w,h)
        ship_dimensions.append((length, width))
ship_dimensions

            
        
           

# functions

In [ ]:
def apply_watershed(image):
    # Convert the image to grayscale
    gray = cv.cvtColor(image, cv.COLOR_RGB2GRAY)
    # Convert the image to binary using Otsu's thresholding
    _, binary = cv.threshold(gray, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
    # Perform distance transform
    dist_transform = cv.distanceTransform(binary, cv.DIST_L2, 5)
    # Normalize the distance image
    dist_transform = cv.normalize(dist_transform, None, 0, 1.0, cv.NORM_MINMAX)
    # Threshold the distance image
    _, sure_fg = cv.threshold(dist_transform, 0.5 * dist_transform.max(), 255, 0)
    sure_fg = np.uint8(sure_fg)
    unknown = cv.subtract(binary, sure_fg)
    # Label the foreground objects
    _, markers = cv.connectedComponents(sure_fg)
    markers = markers + 1
    markers[unknown == 255] = 0
    # Apply the Watershed algorithm
    markers = cv.watershed(image, markers)
    return markers

 

In [ ]:
def measure_ships(markers):
    # Initialize a list to store ship dimensions
    ship_dimensions = []
    for marker in np.unique(markers):
        if marker == -1 or marker == 1:
            continue
        mask = np.zeros_like(markers, dtype=np.uint8)
        mask[markers == marker] = 255
        contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        for contour in contours:
            x, y, w, h = cv.boundingRect(contour)
            length = max(w, h)
            width = min(w, h)
            ship_dimensions.append((length, width))
    return ship_dimensions


measure_ships(apply_watershed(img))

In [ ]:
import json
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Load the JSON file
with open('/kaggle/input/ships-in-satellite-imagery/shipsnet.json') as f:
    dataset = json.load(f)

data = np.array(dataset['data'])
labels = np.array(dataset['labels'])
scene_ids = np.array(dataset['scene_ids'])
locations = np.array(dataset['locations'])




In [ ]:
def preprocess_image(image):
    # Convert the image to grayscale
    gray = cv.cvtColor(image, cv.COLOR_RGB2GRAY)
    # Apply Gaussian Blur to reduce noise and improve contour detection
    blurred = cv.GaussianBlur(gray, (5, 5), 0)
    return blurred

In [ ]:
def classify_ships(ship_dimensions):
    classifications = []
    for length, width in ship_dimensions:
        if length < 20:
            classification = 'A'
        elif length < 40:
            classification = 'B'
        elif length < 60:
            classification = 'C'
        else:
            classification = 'D'
        classifications.append(classification)
    return classifications


In [ ]:
images = images.astype(np.uint8)


In [ ]:
import numpy as np

def reshape_to_rgb(data):
    """
    Reshapes a list of 19200 integers representing an 80x80 RGB image
    into a 3D numpy array with shape (80, 80, 3).

    Parameters:
    data (list of int): The list of 19200 integers representing the image.

    Returns:
    np.ndarray: A 3D numpy array representing the RGB image.
    """
    if len(data) != 19200:
        raise ValueError("The input data must be a list of 19200 integers.")

    # Number of pixels in each channel
    num_pixels = 80 * 80

    # Separate the channels
    red_channel = data[:num_pixels]
    green_channel = data[num_pixels:2*num_pixels]
    blue_channel = data[2*num_pixels:]

    # Reshape each channel to 80x80
    red_channel = np.array(red_channel).reshape((80, 80))
    green_channel = np.array(green_channel).reshape((80, 80))
    blue_channel = np.array(blue_channel).reshape((80, 80))

    # Stack the channels to get the final image
    image = np.stack((red_channel, green_channel, blue_channel), axis=-1)

    return image

# Assuming data is a list of lists, where each inner list is of length 19200
# data = [your_data_list1, your_data_list2, ...]  # Replace with your actual data lists



In [ ]:
# Reshape each element in the data list if necessary
for i in range(len(data)):
    if isinstance(data[i], list) and len(data[i]) == 19200:
        data[i] = reshape_to_rgb(data[i])

print(data[0].shape)  # Should print (80, 80, 3)


In [ ]:
data[0].shape